In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.constants import h, c, eps0, a0

#from light import GaussianBeam

<h3>Frequency for cooler laser</h3>

In [2]:
k39_D2 = (c / (766.700921822*u.nm)).to(u.THz)
k40_D2 = (c / (766.700674872*u.nm)).to(u.THz)

In [3]:
RM1 = k40_D2 - 50*u.MHz + 571.5*u.MHz
RM2 = k40_D2 - 50*u.MHz - 714.3*u.MHz

(RM1 - RM2).to(u.MHz)

<Quantity 1285.80000001 MHz>

In [4]:
laser_freq1 = RM1 - 2*80*u.MHz - 2*200*u.MHz
laser_freq2 = RM2 + 2*350*u.MHz 

(laser_freq1 - laser_freq2).to(u.MHz)

<Quantity 25.8 MHz>

In [5]:
offset = (0.01622*u.THz-0.0337*u.THz).to(u.GHz)

In [6]:
offset

<Quantity -17.48 GHz>

<h3>K39 cross over D1 line vs K40 D1 line</h3>

In [7]:
k39_D1 = (c / (770.108385049*u.nm).to(u.m)).to(u.THz)
k40_D1 = (c / (770.108136507*u.nm).to(u.m)).to(u.THz)

vapor_cell_lines = {
    '1->1'  : k39_D1 + 288.6*u.MHz - 34.7*u.MHz,
    '1->CO' : k39_D1 + 288.6*u.MHz - 6.95*u.MHz,
    '1->2'  : k39_D1 + 288.6*u.MHz + 20.8*u.MHz,
    'CO->1' : k39_D1 + 57.75*u.MHz - 34.7*u.MHz,
    'CO->CO': k39_D1 + 57.75*u.MHz - 6.95*u.MHz,
    'CO->2' : k39_D1 + 57.75*u.MHz + 20.8*u.MHz,
    '2->1'  : k39_D1 - 173.1*u.MHz - 34.7*u.MHz,
    '2->CO' : k39_D1 - 173.1*u.MHz - 6.95*u.MHz,
    '2->2'  : k39_D1 - 173.1*u.MHz + 20.8*u.MHz,
}

for key, value in vapor_cell_lines.items():
    print(key, value)

1->1 389.28631261617005 THz
1->CO 389.28634036617007 THz
1->2 389.2863681161701 THz
CO->1 389.28608176617007 THz
CO->CO 389.2861095161701 THz
CO->2 389.2861372661701 THz
2->1 389.2858509161701 THz
2->CO 389.2858786661701 THz
2->2 389.2859064161701 THz


In [5]:
vapor_cell_lines['2->2']-offset

<Quantity 389.30338642 THz>

In [6]:
vapor_cell_lines['CO->CO']-offset

<Quantity 389.30358952 THz>

In [7]:
k40_D1-offset

<Quantity 389.30366435 THz>

In [8]:
# I find the signal at with:
# T = 20.6°C
# I = 174.10125mA
# V = 94V
389.30307*u.THz

<Quantity 389.30307 THz>

In [9]:
print('K39 D1 line: ',k39_D1)
print('K40 D1 line: ', k40_D1)
print('Difference K39 D1 - K40 D1: ', (k39_D1 - k40_D1).to(u.MHz))
print('Difference vapor cell D1 lines - K40 D1: ')
for key, value in vapor_cell_lines.items():
    print(key, (value - k40_D1).to(u.MHz))

K39 D1 line:  389.2860587161701 THz
K40 D1 line:  389.28618435298785 THz
Difference K39 D1 - K40 D1:  -125.63681775645819 MHz
Difference vapor cell D1 lines - K40 D1: 
1->1 128.26318220504618 MHz
1->CO 156.0131822202493 MHz
1->2 183.7631822354524 MHz
CO->1 -102.58681777486345 MHz
CO->CO -74.83681775966033 MHz
CO->2 -47.08681774445722 MHz
2->1 -333.4368177547731 MHz
2->CO -305.68681773956996 MHz
2->2 -277.93681772436685 MHz


<h3>Stark shift of K40 D1 line</h3>

In [10]:
def dE_Stark(alpha: float, E: u.Quantity,) -> u.Quantity:
    """Calculate Stark shift in MHz for a given electric field and polarizability.
    
    Args:
        alpha: polarizability in atomic units as float
        E: electric field in SI units as astropy.units.Quantity
        
    Returns:
        Stark shift in MHz as astropy.units.Quantity
        
    """
    return (-0.5 * 4*np.pi*eps0*a0**3/h * alpha * E**2).to(u.MHz)

In [11]:
hlattice_x_1 = GaussianBeam(
    beam_direction=[1, 0, 0],
    pol='linear vertical', # vertical polarization for bow-tie beams
    lambda_=1064*u.nm,
    w0=350*u.um,
    P=11*u.W,
)

hlattice_x_2 = GaussianBeam(
    beam_direction=[-1, 0, 0],
    pol='linear vertical', # vertical polarization for bow-tie beams
    lambda_=1064*u.nm,
    w0=350*u.um,
    P=11*u.W,
)

hlattice_y_1 = GaussianBeam(
    beam_direction=[0, 1, 0],
    pol='linear vertical', # vertical polarization for bow-tie beams
    lambda_=1064*u.nm,
    w0=350*u.um,
    P=11*u.W,
)

hlattice_y_2 = GaussianBeam(
    beam_direction=[0, -1, 0],
    pol='linear vertical', # vertical polarization for bow-tie beams
    lambda_=1064*u.nm,
    w0=350*u.um,
    P=11*u.W,
)

lambda_50GHz_detuned_from_D2 = (c / (c/(766.700674872*u.nm).to(u.m) - 50*u.GHz)).to(u.nm)

cooler_x = GaussianBeam(
    beam_direction=[-1, 0, 0],
    pol='linear horizontal', # polarization should be linear and parallel to substrate
    lambda_=lambda_50GHz_detuned_from_D2,
    w0=1*u.mm,
    I0=(0.79+0.47+0.49)*(u.W / u.cm**2)
)

cooler_y = GaussianBeam(
    beam_direction=[0, -1, 0],
    pol='linear horizontal', # polarization should be linear and parallel to substrate
    lambda_=lambda_50GHz_detuned_from_D2,
    w0=1*u.mm,
    I0=2*(u.W / u.cm**2)
)

lattice_beams = [hlattice_x_1, hlattice_x_2, hlattice_y_1, hlattice_y_2,] 
cooler_beams = [cooler_x, cooler_y,]

In [12]:
repumper_F = GaussianBeam(
    beam_direction=[0, -1, 0],
    pol='linear horizontal', # polarization should be linear and parallel to substrate
    lambda_=770.108136507*u.nm,
    w0=1*u.mm,
    I0=5.8*(u.mW / u.cm**2)
)

repumper_mF = GaussianBeam(
    beam_direction=[0, -1, 0],
    pol='linear horizontal', # polarization should be linear and parallel to substrate
    lambda_=770.108136507*u.nm,
    w0=1*u.mm,
    I0=1.6*(u.mW / u.cm**2)
)

In [13]:
repumper_F.P

<Quantity 9.1106187e-05 W>

In [14]:
repumper_mF.P

<Quantity 2.51327412e-05 W>

In [15]:
cooler_y.P

<Quantity 0.03141593 W>

In [16]:
cooler_x.P

<Quantity 0.02748894 W>

In [17]:
lambda_50GHz_detuned_from_D2

<Quantity 766.79872689 nm>

In [18]:
def interpolate(lambda_1, alpha_1, lambda_2, alpha_2, lambda_target):
    return alpha_1 + (alpha_2 - alpha_1) * (lambda_target - lambda_1) / (lambda_2 - lambda_1) 

In [19]:
alpha_k40_4S_1_2_1064 = 599.16
alpha_k40_4S_1_2_767 =  interpolate(lambda_1=766.78200, alpha_1=883200.52, lambda_2=766.80200, alpha_2=706297.75, lambda_target=lambda_50GHz_detuned_from_D2.value)
alpha_k40_4P_1_2_1064 = -3151.72
alpha_k40_4P_1_2_767 = 10474.68

In [20]:
E = 0*u.V/u.m
for beam in lattice_beams:
    E = E + beam.E_field_amplitude(0*u.m, 0*u.m, 0*u.m)
    
dE_4S_1_2_due_to_lattice = dE_Stark(alpha_k40_4S_1_2_1064, E).real # is real anyways, just get rid of + 0j
dE_4P_1_2_due_to_lattice = dE_Stark(alpha_k40_4P_1_2_1064, E).real # is real anyways, just get rid of + 0j

print('Stark shift of 4S_1/2 state due to lattice: ', dE_4S_1_2_due_to_lattice)
print('Stark shift of 4P_1/2 state due to lattice: ', dE_4P_1_2_due_to_lattice)

Stark shift of 4S_1/2 state due to lattice:  -5.137313213264527 MHz
Stark shift of 4P_1/2 state due to lattice:  27.023454170021484 MHz


In [21]:
E = 0*u.V/u.m
for beam in cooler_beams:
    E = E + beam.E_field_amplitude(0*u.m, 0*u.m, 0*u.m)

dE_4S_1_2_due_to_cooler = dE_Stark(alpha_k40_4S_1_2_767, E).real # is real anyways, just get rid of + 0j
dE_4P_1_2_due_to_cooler = dE_Stark(alpha_k40_4P_1_2_767, E).real # is real anyways, just get rid of + 0j

print('Stark shift of 4S_1/2 state due to cooler: ', dE_4S_1_2_due_to_cooler)
print('Stark shift of 4P_1/2 state due to cooler: ', dE_4P_1_2_due_to_cooler)

Stark shift of 4S_1/2 state due to cooler:  -0.5163554985943527 MHz
Stark shift of 4P_1/2 state due to cooler:  -0.0073562283576365005 MHz


In [22]:
dE_4S_1_2 = dE_4S_1_2_due_to_lattice + dE_4S_1_2_due_to_cooler
dE_4P_1_2 = dE_4P_1_2_due_to_lattice + dE_4P_1_2_due_to_cooler

print('Total Stark shift of 4S_1/2 state: ', dE_4S_1_2)
print('Total Stark shift of 4P_1/2 state: ', dE_4P_1_2)

Total Stark shift of 4S_1/2 state:  -5.653668711858879 MHz
Total Stark shift of 4P_1/2 state:  27.016097941663848 MHz


In [23]:
Stark_shifted_k40_D1 = k40_D1 - dE_4S_1_2 + dE_4P_1_2

print("Stark shifted k40 D1 line: ", Stark_shifted_k40_D1)

Stark shifted k40 D1 line:  389.2862170227545 THz


<h3>Difference of F- and m_F-repumper transitions to K39 cross over D1 line</h3>

In [24]:
F_transition = Stark_shifted_k40_D1 - 714.3*u.MHz - 69.0*u.MHz - 80.0*u.MHz
m_F_transition = Stark_shifted_k40_D1 + 571.5*u.MHz + 86.3*u.MHz - 80.0*u.MHz

print('F transition: ', F_transition)
print('m_F transition: ', m_F_transition)
print('Difference F transition - m_F transition: ', (F_transition - m_F_transition).to(u.MHz))

F transition:  389.28535372275445 THz
m_F transition:  389.2867948227545 THz
Difference F transition - m_F transition:  -1441.1000000222884 MHz


In [25]:
laser_frequency_1 = F_transition + 2*80*u.MHz + 2*200*u.MHz
laser_frequency_2 = m_F_transition - 2*80*u.MHz - 2*350*u.MHz
unwanted_laser_difference = (laser_frequency_1 - laser_frequency_2).to(u.MHz)

print('Laser frequency 1: ', laser_frequency_1)
print('Laser frequency 2: ', laser_frequency_2)
print('Difference laser frequency 1 - laser frequency 2: ', unwanted_laser_difference)

Laser frequency 1:  389.28591372275446 THz
Laser frequency 2:  389.2859348227545 THz
Difference laser frequency 1 - laser frequency 2:  -21.100000026308408 MHz


In [26]:
# Let's say we change the AOM's to achieve the correct F-m_F difference in such a way that we correct the F-transition
# with -x * unwanted_laser_difference and the m_F-transition with (1-x) * unwanted_laser_difference for x in [-1,1]
x = 7.3065843366748595*u.MHz / unwanted_laser_difference
laser_frequency_1_corrected = laser_frequency_1 - x * unwanted_laser_difference
laser_frequency_2_corrected = laser_frequency_2 + (1-x) * unwanted_laser_difference
unwanted_laser_difference_corrected = (laser_frequency_1_corrected - laser_frequency_2_corrected).to(u.MHz)

print('Difference laser frequency 1 corrected - laser frequency 2 corrected: ', unwanted_laser_difference_corrected)
print('Shift of F-repumper arm: ', -x*unwanted_laser_difference)
print('Shift of m_F-repumper arm: ', (1-x)*unwanted_laser_difference)

Difference laser frequency 1 corrected - laser frequency 2 corrected:  0.0 MHz
Shift of F-repumper arm:  -7.3065843366748595 MHz
Shift of m_F-repumper arm:  -28.406584362983267 MHz


In [27]:
laser_frequency = laser_frequency_1_corrected

print('Laser frequency: ', laser_frequency)
print('Difference K39 vapor cell D1 lines - laser frequency: ')
for key, value in vapor_cell_lines.items():
    print(key, (value - laser_frequency).to(u.MHz))

Laser frequency:  389.2859064161701 THz
Difference K39 vapor cell D1 lines - laser frequency: 
1->1 406.199999929413 MHz
1->CO 433.94999994461614 MHz
1->2 461.69999995981925 MHz
CO->1 175.3499999495034 MHz
CO->CO 203.0999999647065 MHz
CO->2 230.84999997990963 MHz
2->1 -55.50000003040623 MHz
2->CO -27.750000015203113 MHz
2->2 0.0 MHz
